# 2. DeepLC

In [3]:
!pip install deeplc deeplcretrainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.2/32.2 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.2/522.2 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.2/236.2 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.9/52.9 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
# Import default libs
import re
import os

# Import data libs
import pandas as pd

# Import DeepLC
from deeplc import DeepLC
from deeplc import FeatExtractor

# Import plotting libs
from matplotlib import pyplot as plt
import seaborn as sns

# Supress warnings (or at least try...)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Set the dir of analysis
main_dir = "DeepLC_data/"

# 2.0 Introduction

## 2.0.1 High Performance/Pressure Liquid Chromatography
As all data acquisition parts in LC-IM-MS the LC part separates analytes. In this case the separation is based on the physicochemical properties of our peptides. In most cases peptides are separated based on their hydrophobicity in so-called reverse-phase LC. A more detailed explanation is provided here: https://www.ssi.shimadzu.com/products/liquid-chromatography/knowledge-base/hplc-basics.html

This schematic representation of the instrument is the nightmare of every mass spectrometrist (i.e., this instrument is very prown to break):

![workflow_lc](images/nightmare.gif)

![workflow_lc](images/LC_workflow.png)

Source: https://www.technologynetworks.com/analysis/articles/liquid-chromatography-including-hplc-uhplc-and-lcxlc-344048



## 2.0.2 Mobile and stationary phase; migration through the column
In HPLC the separation happens in a column:

<img src="images/column.jpg" alt="column" width="200"/>

This separation is achieved with two phases, the stationary phase can look like this:
<img src="images/stat_phase.jpg" alt="stationary" width="200"/>

The sample is pumped the column with two solvents that form the mobile phase (A and B):
<img src="images/mob_phase.png" alt="mobile" width="200"/>

The analytes in the sample interact with both the stationary and mobile phase. The physicochemical properties of the peptides dictate how much they can interact with either of the two phases. (More) interaction with the mobile phase results in migration of the peptides in the column:
![workflow_lc](images/sample-bands-animation.gif)

Source: https://www.ssi.shimadzu.com/products/liquid-chromatography/knowledge-base/hplc-basics.html

# 2.1 DeepLC predictions

## 2.1.1 Reading data and preparing instances of DeepLC objects

First we will read a table that contains all of our data:

In [2]:
df = pd.read_csv("https://dl.dropboxusercontent.com/s/bok4w3jw2gxohbz/deeplc_input.csv",index_col=0)

The data looks like this:

In [3]:
df

,scan,best_psm,database_peptide,modifications,precursor_mass,peptide_mass,rt,q_value
2,4,1,AGAVVGK,NaN,599.362768,600.359497,0.765235,0.706835
3,4,0,GAAAMGK,NaN,599.362768,604.300232,0.765235,0.768968
11,209,0,SAAAAAR,NaN,616.327734,616.329224,28.547861,0.679285
12,209,0,TQGSAAPGSK,3|semi_tryptic,616.327734,616.318054,28.547861,0.551056
13,209,1,GSETPGATPGSK,5|semi_tryptic,616.327734,616.318054,28.547861,0.089244
...,...,...,...,...,...,...,...,...
4452830,128083,1,SIEDYSQSIHTRTISGSPR,17|ethylamino,2160.093481,2160.092529,10516.730400,0.505389
4452832,128083,0,SIAASRFAPHIEKCIGMGR,8|Ala->Met|14|carbamidomethyl,2160.093481,2160.075684,10516.730400,0.668446
4452833,128083,0,SIAGFVASINEGMTRWFSR,13|Met->Tyr,2160.093481,2160.075195,10516.730400,0.516400
4452834,128083,0,SIAEAAMYWIVNWPRPQR,17|Gln->Thr,2160.093481,2160.093994,10516.730400,0.645799


We have multiple columns describing the scan number, if a PSM is the best ranked PSM, if there are any modifications, the precursor mass, the peptide mass, the observed retention time, and the associated q-value with the PSM.

For DeepLC we need strings instead of NaN in the modifications column, so lets replace those with empty strings:

In [4]:
df.fillna("",inplace=True)

Here we sample rows from the original table, this is purely done for computational reasons (e.g., on a laptop). Feel free to increase the numbers if you have a faster system.

In [5]:
num_total_rows_select = 5000
num_calib = 250

sub_df_pred = df[df["scan"].isin(list(set(df["scan"].sample(num_total_rows_select))))]
sub_df_calib = sub_df_pred[sub_df_pred["scan"].isin(list(set(sub_df_pred[sub_df_pred["q_value"] < 0.01]["scan"].sample(num_calib))))]

In [6]:
sub_df_pred

,scan,best_psm,database_peptide,modifications,precursor_mass,peptide_mass,rt,q_value
327,128923,0,IPIPSPSICQIIR,9|carbamidomethyl,1498.687371,1492.843628,10762.4886,0.645183
328,128923,0,PISTTICPEGIRR,7|carbamidomethyl,1498.687371,1498.792603,10762.4886,0.685492
329,128923,1,HGIVEDWDIMER,,1498.687371,1498.687378,10762.4886,0.006179
416,128938,1,TITGKTITIEVEPSDTIENVK,5|ICPL_13C(6)2H(4),2402.264245,2402.282471,10763.5308,0.000027
717,128982,0,IHVRVHTK,4|Delta:H(2)C(3)O(1),1042.593804,1042.603516,10766.4378,0.745204
...,...,...,...,...,...,...,...,...
4452711,128065,0,EFSEIAK,,819.368566,822.412354,10515.3738,0.698395
4452712,128065,0,ECKAIAK,2|carbamidomethyl,819.368566,818.432007,10515.3738,0.696232
4452713,128065,0,EISTGIAK,,819.368566,817.454529,10515.3738,0.742331
4452714,128065,0,EPPAPIAK,,819.368566,821.464722,10515.3738,0.762207


Here we make sure that the column names are changed to something that DeepLC recognizes:

In [7]:
sub_df_pred.rename({
    "database_peptide" : "seq",
    "rt" : "tr"
},axis=1,inplace=True)

sub_df_calib.rename({
    "database_peptide" : "seq",
    "rt" : "tr"
},axis=1,inplace=True)

Make a feature extraction object. This step can be skipped if you want to use the default feature extraction in DeepLC. This does allow us to select for example a different file for the modifications we want to incorperate.

In [14]:
f_extractor = FeatExtractor(
                            main_path=main_dir,
                            # lib_path_mod=os.path.join(main_dir, "unimod/"),
                            # lib_path_prot_scale=os.path.join(main_dir, "expasy/"),
                            # lib_aa_composition=os.path.join(main_dir, "aa_comp_rel.csv"),
                            # lib_path_smiles=os.path.join(main_dir, "mod_to_smiles/"),
                            # lib_three_to_one=os.path.join(main_dir, "expasy/three_to_one.csv"),
                            add_sum_feat=False,
                            ptm_add_feat=False,
                            ptm_subtract_feat=False,
                            # standard_feat=False,
                            # chem_descr_feat=False,
                            add_comp_feat=False,
                            cnn_feats=True,
                            verbose=True
)

Initiate a DeepLC instance that will perform the calibration and predictions:

In [17]:
dlc = DeepLC(
    cnn_model=True,
    f_extractor=f_extractor,
    pygam_calibration=False,
    verbose=True
)

We need to calibrate predictions to our specific LC setup to make them valid, so first we will feed some predictions to our DeepLC instance:

In [18]:
dlc.calibrate_preds(seq_df=sub_df_calib[sub_df_calib["best_psm"]==1])

1/1 [==============================] - 2s 2s/step


1/1 [==============================] - 2s 2s/step


1/1 [==============================] - 2s 2s/step


1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 1s 1s/step


{}

You can safely ignore most errors and warnings
<img src="images/error.png" alt="mobile" width="200"/>

In [19]:
a=[]
#a.append(sub_df_pred["scan"])
#a=set(a)
a=sub_df_pred["scan"].values
a=list(set(a))
len(a)

4901

## 2.1.2 Is calibration a necessity?

Lets run our mode to generate predictions without calibration:

In [20]:
preds_uncal = dlc.make_preds(seq_df=sub_df_calib, calibrate=False)
sub_df_calib["preds_uncal"] = preds_uncal

4/4 [==============================] - 4s 948ms/step


And lets also make calibrated predictions for the comparison:

In [21]:
preds_cal = dlc.make_preds(seq_df=sub_df_calib)
sub_df_calib["preds"] = preds_cal

4/4 [==============================] - 5s 961ms/step


Now for the comparison of the two:

In [22]:
sub_df_calib_lowerrank = sub_df_calib[sub_df_calib["best_psm"]==0]
sub_df_calib_firstrank = sub_df_calib[sub_df_calib["best_psm"]==1]

# Init plot
plt.figure(figsize=(10,11.5))
ax = plt.gca()
#ax.set_aspect('equal')

# Plot data
plt.scatter(sub_df_calib_firstrank["tr"],sub_df_calib_firstrank["preds_uncal"],s=3.5, alpha=0.5,color="tab:orange")
plt.xlabel("Observed retention time (s)")
plt.ylabel("Predicted retention time (not s...)")

print("Uncalibrated predictions: ")
plt.show()

# Init plot
plt.figure(figsize=(10,10))
ax = plt.gca()
ax.set_aspect('equal')

print("Calibrated predictions: ")
plt.scatter(sub_df_calib_firstrank["tr"],sub_df_calib_firstrank["preds"],s=3.5, alpha=0.5,color="tab:orange")
plt.plot([1500,14500],[1500,14500],c="black",linestyle="dotted")
plt.xlabel("Observed retention time (s)")
plt.ylabel("Predicted retention time (s)")

plt.show()

Uncalibrated predictions: 
Calibrated predictions: 


### 2.1.3 Questions - calibration

<ol>
  <li>Is calibration required? For what down-stream analysis would we need calibration? For which is it not required?</li>
  <li>What would be a good method for selecting calibration peptides?</li>
  <li>Do iRT peptides solve the calibration issues?</li>
  <li>Are there other things regarding calibration you want to discuss? Like how it should probably be called alignment.</li>
</ol>

# 2.2 Prediction errors of (ranked) PSMs

## 2.2.1 Comparing rank 1 and lower ranked PSMs for the calibration set

In [23]:
# Init plot
plt.figure(figsize=(10,10))
ax = plt.gca()
ax.set_aspect('equal')

# Plot data
plt.scatter(sub_df_calib_lowerrank["tr"],sub_df_calib_lowerrank["preds"],s=3.5, alpha=0.5,label="Lower rank PSM")
plt.scatter(sub_df_calib_firstrank["tr"],sub_df_calib_firstrank["preds"],s=3.5, alpha=0.5,label="First ranked PSM")
plt.plot([1500,14500],[1500,14500],c="black",linestyle="dotted")

plt.xlabel("Observed retention time (s)")
plt.ylabel("Predicted retention time (s)")
plt.legend()

plt.show()

# Init plot
plt.figure(figsize=(10,10))
ax = plt.gca()
ax.set_aspect('equal')

# Plot data
plt.scatter(sub_df_calib_lowerrank["tr"],sub_df_calib_lowerrank["preds"],s=3.5, alpha=0.5, color="tab:blue",label="Lower rank PSM")
plt.plot([1500,14500],[1500,14500],c="black",linestyle="dotted")

plt.xlabel("Observed retention time (s)")
plt.ylabel("Predicted retention time (s)")
plt.legend()

plt.show()

# Init plot
plt.figure(figsize=(10,10))
ax = plt.gca()
ax.set_aspect('equal')

# Plot data
plt.scatter(sub_df_calib_firstrank["tr"],sub_df_calib_firstrank["preds"],s=3.5, alpha=0.5, color="tab:orange",label="First ranked PSM")
plt.plot([1500,14500],[1500,14500],c="black",linestyle="dotted")

plt.xlabel("Observed retention time (s)")
plt.ylabel("Predicted retention time (s)")
plt.legend()

plt.show()

## 2.2.2 Comparing rank 1 and lower ranked PSMs for the whole data set

Lets first make predictions:

In [24]:
preds = dlc.make_preds(seq_df=sub_df_pred)
sub_df_pred["preds"] = preds

76/76 [==============================] - 97s 1s/step


In [35]:
sub_df_pred_lowerrank = sub_df_pred[sub_df_pred["best_psm"]==0]
sub_df_pred_firstrank = sub_df_pred[sub_df_pred["best_psm"]==1]

# Init plot
plt.figure(figsize=(10,10))
ax = plt.gca()
ax.set_aspect('equal')

# Plot data
plt.scatter(sub_df_pred_lowerrank["tr"],sub_df_pred_lowerrank["preds"],s=3.5, alpha=0.25,label="Lower rank PSM")
plt.scatter(sub_df_pred_firstrank["tr"],sub_df_pred_firstrank["preds"],s=3.5, alpha=0.25,label="First ranked PSM")
plt.plot([1500,14500],[1500,14500],c="black",linestyle="dotted")

plt.xlabel("Observed retention time (s)")
plt.ylabel("Predicted retention time (s)")
plt.legend()

plt.show()

# Init plot
plt.figure(figsize=(10,10))
ax = plt.gca()
ax.set_aspect('equal')

# Plot data
plt.scatter(sub_df_pred_lowerrank["tr"],sub_df_pred_lowerrank["preds"],s=3.5, alpha=0.5, color="tab:blue",label="Lower rank PSM")
plt.plot([1500,14500],[1500,14500],c="black",linestyle="dotted")

plt.xlabel("Observed retention time (s)")
plt.ylabel("Predicted retention time (s)")
plt.legend()

plt.show()

# Init plot
plt.figure(figsize=(10,10))
ax = plt.gca()
ax.set_aspect('equal')

# Plot data
plt.scatter(sub_df_pred_firstrank["tr"],sub_df_pred_firstrank["preds"],s=3.5, alpha=0.5, color="tab:orange",label="First ranked PSM")
plt.plot([1500,14500],[1500,14500],c="black",linestyle="dotted")

plt.xlabel("Observed retention time (s)")
plt.ylabel("Predicted retention time (s)")
plt.legend()

plt.show()

## 2.2.2 Questions - prediction errors of (ranked) PSMs

<ol>
  <li>Is there a difference in error between different ranked PSMs?</li>
  <li>What do you think the error distributions look like of these two classes?</li>
  <li>Why are there both horizontal and vertical streeks visible?</li>
</ol>

Execute the following cell to see the error distributions:

In [26]:
sns.distplot(sub_df_pred_lowerrank["tr"]-sub_df_pred_lowerrank["preds"],
             hist = False, kde = True, label="Lower rank PSM")
sns.distplot(sub_df_pred_firstrank["tr"]-sub_df_pred_firstrank["preds"],
             hist = False, kde = True, label="First ranked PSM")
plt.xlabel("Error (s)")
plt.show()

# 2.3 Predict retention times of modified peptides

## 2.3.1 Effect of modifications on retention time

In [27]:
def plot_modification(sub_df_best,modification="carbamidomethyl"):
    # Init plot
    plt.figure(figsize=(7,7))
    ax = plt.gca()
    ax.set_aspect('equal')

    # Plot data
    plt.scatter(sub_df_best[sub_df_best["modifications"].str.contains(modification)]["tr"],sub_df_best[sub_df_best["modifications"].str.contains(modification)]["preds"],alpha=0.5,s=4)
    plt.plot([1500,14500],[1500,14500],c="black",linestyle="dotted")

    plt.title(modification)
    plt.xlabel("Observed retention time (s)")
    plt.ylabel("Predicted retention time (s)")

    plt.show()

In [28]:
sub_df_best = sub_df_pred[sub_df_pred["best_psm"]==1]
sub_df_best = sub_df_best[sub_df_best["q_value"]<0.001]

plot_modification(sub_df_best,modification="carbamidomethyl")
plot_modification(sub_df_best,modification="Formyl")
plot_modification(sub_df_best,modification="Deamidated")
plot_modification(sub_df_best,modification="Dehydrated")
plot_modification(sub_df_best,modification="Ammonium")
plot_modification(sub_df_best,modification="Sulfide")

## 2.3.2 Questions - retention time prediction of modified peptides

<ol>
  <li>Would it be hard for a model to predict retention times of modifications that was not trained on?</li>
  <li>What modifications would be hardest?</li>
</ol>

# 2.4 Playground - design your own peptides and modifications and predict their retention time

## 2.4.1 Make predictions for your own peptide and modifications combos

Provide the data for peptides you want to predict:

In [29]:
#IIVINTPNNPIGK
dict_effect_aa = {
    "seq" : ["IIVINKPNNPIGK", "IIVINTPNNPIGK", "IIVINAPNNPIGK", "IIVINWPNNPIGK"],
    "modifications" : ["","","",""],
    "tr" : [0,1,2,3]
}

df_effect_aa = pd.DataFrame(dict_effect_aa)

In [30]:
preds = dlc.make_preds(seq_df=df_effect_aa)

1/1 [==============================] - 1s 713ms/step


Lets have a look at their predictions:

In [31]:
plt.scatter(df_effect_aa.index,preds)
plt.xticks(df_effect_aa.index,df_effect_aa["seq"])
plt.ylabel("Predicted retention time (s)")
plt.show()

Provide the data for peptides+modifications you want to predict:

In [32]:
#IIVINTPNNPIGK
dict_effect_aa = {
    "seq" : ["IIVINCPNNPIGK", "IIVINCPNNPIGK", "IIVINQPNNPIGK", "IIVINQPNNPIGK", "IIVINMPNNPIGK", "IIVINMPNNPIGK"],
    "modifications" : ["","6|carbamidomethyl","","6|Deamidated","","6|Formyl"],
    "tr" : [0,1,2,3,4,5]
}

df_effect_aa = pd.DataFrame(dict_effect_aa)

In [33]:
preds = dlc.make_preds(seq_df=df_effect_aa)

1/1 [==============================] - 1s 504ms/step


In [34]:
plt.scatter(df_effect_aa.index,preds)
plt.xticks(df_effect_aa.index,df_effect_aa["seq"]+"+"+df_effect_aa["modifications"],rotation=90)
plt.ylabel("Predicted retention time (s)")
plt.show()

## 2.4.2 Questions - playground retention time prediction

<ol>
  <li>Can you design a peptide that falls in between "IIVINKPNNPIGK" and "IIVINTPNNPIGK" in terms of retention time?</li>
  <li>What effect do certain modifications have? Is this expected?</li>
  <li>Do you expect that modifications always have the same effect?</li>
</ol>